In [1]:
import torch
from pathlib import Path
from tqdm import tqdm
import json

In [2]:
from pytorch_transformers import BertTokenizer, BertForMaskedLM

In [3]:
crossencoder_eval_foldername = "crossencoder1212693-1214840-eval"

In [ ]:
#extract from the crossencoder_eval_foldername the first job_id
bienc_job_id = crossencoder_eval_foldername.split("-")[0].replace("crossencoder", "")
#bienc_job_id="1212693"
bienc_job_id

In [5]:
cross_encoder_eval_path = Path("models") / "BLINK_MODELS" / crossencoder_eval_foldername

In [ ]:
eval_results=json.load((cross_encoder_eval_path / "eval_results.json").open())
eval_results.keys()

In [ ]:
for k,v in eval_results.items():
    if isinstance(v, list):
        print(k, len(v))
    else:
        print(k, v)

In [ ]:
bi_encoder_inference = torch.load(f"output/melart_blink/{bienc_job_id}/top30_candidates/test.t7")
bi_encoder_inference.keys()

In [ ]:
len(bi_encoder_inference["candidate_vecs"])

In [ ]:
max(bi_encoder_inference["labels"].flatten())

In [11]:
tokenizer=BertTokenizer.from_pretrained("bert-large-uncased")

In [ ]:
tokenizer.decode(bi_encoder_inference["candidate_vecs"][0][0].tolist(), skip_special_tokens=True)

In [ ]:
for i in range(10):
    print(tokenizer.decode(bi_encoder_inference["candidate_vecs"][0][i].tolist(), skip_special_tokens=True))

In [14]:
candidates_txt=open("/hpi/fs00/home/alejandro.sierra/MELArt_experiments/BLINK-main/data/melart_blink/documents/documents.jsonl").read().split("\n")

In [ ]:
decoded_results=[]
for i,context in tqdm(enumerate(bi_encoder_inference["context_vecs"]),total=len(bi_encoder_inference["context_vecs"]),desc="Decoding"):
    sentence=tokenizer.decode(context.tolist(), skip_special_tokens=True)
    entities_ranked=[]
    for j in range(30):
        entities_ranked.append(tokenizer.decode(bi_encoder_inference["candidate_vecs"][i][j].tolist(), skip_special_tokens=True))
    indices=bi_encoder_inference["indices"][i]
    results=[]
    for j,ent in enumerate(entities_ranked):
        candidate=candidates_txt[indices[j]]
        candidate_obj=json.loads(candidate)
        results.append({"entity":ent,"index":indices[j], "document_id":candidate_obj["document_id"]})
    sorted_indices=eval_results["sorted_indices"][i]
    re_ranked_results=[results[j] for j in sorted_indices]
    gt_rank=eval_results["ranks"][i]
    res={"sentence":sentence,"entities_ranked":re_ranked_results, "gt_rank":gt_rank}
    decoded_results.append(res)

In [16]:
with open(cross_encoder_eval_path / "inference.json","w") as f:
    json.dump(decoded_results,f,indent=4)